## _LLM Powered Chatbot_ ##

In [2]:
#Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
#Import Chatmodel
from langchain_groq import ChatGroq
model = ChatGroq(model='gemma2-9b-it',
                 groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f545eaad750>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f545eaaebc0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import AIMessage,HumanMessage
response = model.invoke([
    HumanMessage(content='Hi I am Aswin ,I am a data Engineer with Infosys')
])
response.content

"Hi Aswin! \n\nIt's nice to meet you.  \n\nAs a data engineer at Infosys, what kind of projects are you working on?  Are you using any interesting technologies? I'm always interested in learning more about the work data engineers do.\n"

In [6]:
from langchain_core.messages import AIMessage,HumanMessage
model.invoke([
    HumanMessage(content='Hi I am Aswin ,I am a data Engineer with Infosys'),
    AIMessage(content ="Hi Aswin,\n\nIt's nice to meet you! It's great to hear you're a Data Engineer at Infosys. That's a fascinating field. \n\nWhat kind of data engineering projects are you currently working on?  \n\nI'm always eager to learn more about how data is being used to solve real-world problems.\n"),
    HumanMessage(content="Whats my name and occupation?")
])


AIMessage(content="You told me your name is Aswin and that you are a Data Engineer at Infosys. 😊  \n\nIs there anything else you'd like to chat about?  Perhaps you'd like to discuss data engineering trends or challenges you're facing?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 113, 'total_tokens': 168, 'completion_time': 0.1, 'prompt_time': 0.005387591, 'queue_time': 0.24632072000000002, 'total_time': 0.105387591}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-47263265-fc39-441c-9dad-00c574218b1a-0', usage_metadata={'input_tokens': 113, 'output_tokens': 55, 'total_tokens': 168})

#### _Message History using Langchain_ ####

In [7]:
#Import libraries
#Create the history Store
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
#Wrapper that appends to and replays from `history`
from langchain_core.runnables.history import RunnableWithMessageHistory

In [8]:
store ={}
def get_session_history(session_id:str)->BaseChatMessageHistory:

    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [9]:
with_message_history = RunnableWithMessageHistory(model,get_session_history)
config = {'configurable' :{'session_id':'chat1'}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi I am Aswin ,I am a data Engineer with Infosys')],
    config = config
)
response.content

"It's nice to meet you, Aswin! 👋  \n\nIt's great to know you're a Data Engineer at Infosys. That's a really exciting field! \n\nWhat kind of projects are you working on these days?  Are there any particular technologies or challenges you're particularly interested in?  \n\nI'm always eager to learn more about what people are doing in the world of data engineering. 😄\n"

In [11]:

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)
response.content


'Your name is Aswin.  😊  I remember!  We just met.  \n\n\n\n\n\n\n'

In [12]:
config2 = {'configurable' :{'session_id':'chat2'}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config2
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to use it!\n"

### _ChatPromptTemplate_ ###

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [('system','You are a helpful assistant,Answer all the questions asked'),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [17]:
chain = prompt|model

In [18]:
with_message_history= RunnableWithMessageHistory(chain,
                                                 get_session_history)

In [21]:
config = {'configurable':{'session_id':'chat3'}}

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content='My name is Aswin,I am a data engineer')],
    config = config
)

In [23]:
response.content

"Hi Aswin! It's nice to meet you.\n\nI'm ready to help with any questions you have. Since you're a data engineer, I imagine you have a lot of interesting things to ask about data, technology, or maybe even just need a break and want to chat.\n\nWhat can I do for you today?  \n\n"

##### _Case2 - Parametersing the output prompt template_ #####

In [28]:
prompts = ChatPromptTemplate.from_messages(
    [('system','You are a helpful assistant,answer in {language}'),
    MessagesPlaceholder(variable_name='messages')  
    ]
)

In [29]:
chain = prompts|model

In [31]:
response= chain.invoke(
    {"messages":[HumanMessage(content = 'I am Aswin Pushkar,I am a data engineer')],
     "language":"Hindi"}
)

In [32]:
response.content

'नमस्ते Aswin Pushkar!\n\nमुझे जानकर अच्छा लगा कि आप एक डेटा इंजीनियर हैं। \n\nआप मुझे कैसे मदद कर सकता हूँ?  क्या आपके कोई विशिष्ट प्रश्न हैं या कोई कार्य जिसे मैं आपके लिए कर सकता हूँ? \n\n'

In [33]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)

In [34]:
config = {'configurable':{'session_id':'chat4'}}

In [35]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content = 'I am Aswin Pushkar,I am a data engineer')],
     'language':'Hindi'},
     config = config
)

In [36]:
response.content


'नमस्ते Aswin Pushkar! 😊  मुझे यह जानकर खुशी हो रही है कि आप एक डेटा इंजीनियर हैं।  \n\nआपके लिए मैं कैसे मदद कर सकता हूँ? क्या आप मुझे कुछ डेटा इंजीनियरिंग से जुड़े किसी प्रश्न के बारे में पूछना चाहते हैं, या फिर कुछ और? \n\n'

In [37]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content = 'What is my name?')],
     'language':'Hindi'},
     config = config
)
response.content

'आपका नाम Aswin Pushkar है! 😊 \n'

##### _Managing Conversation History_ #####

In [68]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 45 ,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = 'human' 
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="What is my 1+1?"),
    AIMessage(content='Your 1+1 is 2!')
]

In [69]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my 1+1?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your 1+1 is 2!', additional_kwargs={}, response_metadata={})]

In [71]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
         |prompt
         |model
)

In [72]:
response = chain.invoke(
   { 'messages' : messages + [HumanMessage(content = "What is my favourite icecream")],
    'language' : 'English'}
)

In [73]:
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor.  What's your favorite ice cream?  🍦\n"

In [76]:
response = chain.invoke(
   { 'messages' : messages + [HumanMessage(content = "having fun?")],
    'language' : 'Hindi'}
)

In [77]:
response.content

"I am!  It's fun to help people and answer questions.  \n\nIs there anything else I can help you with? 😊\n"

In [57]:
#wrap in message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [58]:
config = {"configurable" :{"session_id":"chat5"}}

In [61]:
response = with_message_history.invoke(
    {"messages": messages +[HumanMessage(content="Whats my favourite icecream?")],
     "language": "English"},
     config = config,
)

In [62]:
response.content

"As a large language model, I don't have access to any personal information about you, including your favorite ice cream flavor.\n\nWhat's your favorite ice cream? 😊🍦  \n\n"